# Read data

The data was downloaded from the this [kaggle dataset](https://www.kaggle.com/datasets/sahirmaharajj/crime-data-from-2020-to-present-updated-monthly).

Since the original dataset is quite large to upload on github, we will be using a sample of 1,000 rows.

In [1]:
import pandas as pd

df = pd.read_csv("input/crime_data_sample.csv")

df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DR_NO           1000 non-null   int64  
 1   Date Rptd       1000 non-null   object 
 2   DATE OCC        1000 non-null   object 
 3   TIME OCC        1000 non-null   int64  
 4   AREA            1000 non-null   int64  
 5   AREA NAME       1000 non-null   object 
 6   Rpt Dist No     1000 non-null   int64  
 7   Part 1-2        1000 non-null   int64  
 8   Crm Cd          1000 non-null   int64  
 9   Crm Cd Desc     1000 non-null   object 
 10  Mocodes         902 non-null    object 
 11  Vict Age        1000 non-null   int64  
 12  Vict Sex        937 non-null    object 
 13  Vict Descent    937 non-null    object 
 14  Premis Cd       1000 non-null   float64
 15  Premis Desc     1000 non-null   object 
 16  Weapon Used Cd  202 non-null    float64
 17  Weapon Desc     202 non-null    ob

In [2]:
df["AREA NAME"].value_counts()

AREA NAME
Central        62523
77th Street    57392
Pacific        53582
Southwest      51643
Hollywood      48357
Olympic        46459
Southeast      46340
N Hollywood    46078
Newton         45712
Wilshire       44112
Rampart        43225
West LA        42144
Northeast      39463
Van Nuys       38824
West Valley    38605
Harbor         37818
Devonshire     37549
Topanga        37499
Mission        36534
Hollenbeck     34203
Foothill       30381
Name: count, dtype: int64

We have 21 areas in the dataset, which is quite a big number for a categorical column.

We will group this areas into fictional bigger regions.

# Grouping areas into regions

In [11]:
# Created by looking at LA map
regions = {
    "Central": ["Central","Hollywood", "Olympic", "Newton", "Wilshire", "Rampart", "Hollenbeck", "Pacific"],
    "Northeast": ["Northeast", "N Hollywood", "Foothill", "Mission"],
    "South": ["Southwest", "Southeast", "Harbor", "77th Street"],
    "Northwest": ["Van Nuys", "West Valley", "Devonshire", "West LA", "Topanga"]
}

In [9]:
def find_region(area_name):
    for region in regions.keys():
        if area_name in regions[region]:
            return region

In [12]:
df["region"] = df["AREA NAME"].apply(find_region)
df["region"].value_counts()

region
Central      362
South        237
Northwest    215
Northeast    186
Name: count, dtype: int64

The dataset was chosen for the project because it contains a good amount of rows and columns and a date columns that spans through various years.

However, for our project, we needed to create dashboards for a ficticional company, and companies need customers.

For this, we were allowed to create sythetic data.

# Generate customers dataset

In [6]:
# Let's find out the min and max date of occurance, so we keep customers creation data in line with the dataset.
print(f"Min occorance date: {df['DATE OCC'].min()}")
print(f"Max occurance date: {df['DATE OCC'].max()}")

Min occorance date: 01/01/2020 12:00:00 AM
Max occurance date: 12/31/2020 12:00:00 AM


In [8]:
from faker import Faker
import random
from datetime import datetime
from typing import OrderedDict

# Initialize Faker
fake = Faker()
Faker.seed(1234)

We can create customers and customer data in a totally randomized way, but finding trends in these for the report later will be challenging as any discovery would be merely coincidence.

Let's insert some sythetic trends in this data also.

In [13]:
def generate_customer_dataset(num_rows):
    dataset = []
    for _ in range(num_rows):
        customer_id = fake.unique.random_int(min=1, max=999999)

        # Using the dates we found earlier
        registration_date = fake.date_between_dates(date_start=datetime(2020, 1, 1), date_end=datetime(2020, 12, 31))

        # We want to create three clusters of customers that later on can be separated by analysing the region.
        # We are going to assign customers to regions according to proportions
        region = fake.random_element(elements=OrderedDict([("North", 0.3), ("Central", 0.5), ("South", 0.2)]))  
        dataset.append((customer_id, registration_date, region))
    
    return pd.DataFrame(dataset, columns=['customer_id', 'registration_date', 'region'])


In [15]:
# Creating a dataset with 5,104 rows
df_customers = generate_customer_dataset(5104)
df_customers.head()

,customer_id,registration_date,region
0,529746,2020-12-01,North
1,268877,2020-06-04,Central
2,752335,2020-12-05,Central
3,372424,2020-10-04,North
4,26689,2020-02-08,Central


In [16]:
# region is not a column that exists in the original dataset.
# To join customers to the original dataset, we will use area_name, and for that we need to add that column to our customers table
def add_area_to_customer(region):
    if region == "North":
        return fake.random_element(elements=regions["Northeast"]+regions["Northwest"])
    else:
        return fake.random_element(elements=regions[region])

In [19]:
df_customers["area_name"] = df_customers["region"].apply(add_area_to_customer)
df_customers.head()

,customer_id,registration_date,region,area_name
0,529746,2020-12-01,North,Topanga
1,268877,2020-06-04,Central,Wilshire
2,752335,2020-12-05,Central,Rampart
3,372424,2020-10-04,North,Northeast
4,26689,2020-02-08,Central,Rampart


In [20]:
# Drop the region column as this is an implicit trend we are inserting in the data
df_customers_out = df_customers.drop('region', axis=1)

df_customers_out.to_csv('output/customers.csv', index=False)

# Generate payments and ratings datasets

For the customers events, we will create payments and ratings.